In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/3章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ３章　可視化の仕組みを構築する１０本ノック

### ノック２１：店舗を絞り込んで可視化できるようにしてみよう

In [4]:
import pandas as pd
from IPython.display import display, clear_output

m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')
order_data = pd.read_csv('tbl_order_202004.csv')
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

# マスターにないコードに対応した文字列を設定
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'

order_data.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,79339111,49,C26387220,50,67393872,2020-04-01 11:00:00,2020-04-01 11:18:00,1,4144,1,浅草店,TK,東京,東京,お持ち帰り,お支払済
1,18941733,85,C48773811,26,91834983,2020-04-01 11:00:00,2020-04-01 11:22:00,0,2877,2,目黒店,TK,東京,東京,デリバリー,お渡し済
2,56217880,76,C66287421,36,64409634,2020-04-01 11:00:00,2020-04-01 11:15:00,0,2603,2,本郷店,TK,東京,東京,デリバリー,お渡し済
3,28447783,190,C41156423,19,73032165,2020-04-01 11:00:00,2020-04-01 11:16:00,0,2732,2,栃木店,TO,北関東,栃木,デリバリー,お渡し済
4,32576156,191,C54568117,71,23281182,2020-04-01 11:00:00,2020-04-01 11:53:00,0,2987,2,伊勢崎店,GU,北関東,群馬,デリバリー,お渡し済


In [5]:
from ipywidgets import Dropdown

def order_by_store(val):
    clear_output()
    display(dropdown)    
    pick_data = order_data.loc[(order_data['store_name']==val['new']) & (order_data['status'].isin([1, 2]))]
    display(pick_data.head())

store_list = m_store['store_name'].tolist()

dropdown = Dropdown(options=store_list)
dropdown.observe(order_by_store, names='value')
display(dropdown)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib

def graph_by_store(val):
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[(order_data['store_name']==val['new']) & (order_data['status'].isin([1, 2]))]
    temp = pick_data[['order_accept_date', 'total_amount']].copy()  
    temp.loc[:,'order_accept_date'] = pd.to_datetime(temp['order_accept_date'])
    temp.set_index('order_accept_date', inplace=True)
    temp.resample('D').sum().plot()
    
dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names='value')
display(dropdown2)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

### ノック２２：複数店舗の詳細を可視化できるようにしてみよう

In [7]:
from ipywidgets import SelectMultiple

def order_by_multi(val):
    clear_output()
    display(select)
    pick_data = order_data.loc[(order_data["store_name"].isin(val["new"])) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

select = SelectMultiple(options=store_list)
select.observe(order_by_multi, names="value")
display(select)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

In [8]:
def graph_by_multi(val):
    clear_output()
    display(select2)

    fig = plt.figure(figsize=(17,4))
    plt.subplots_adjust(wspace=0.25, hspace=0.6)

    i=0

    for trg in val["new"]:
        pick_data = order_data[(order_data["store_name"]==trg) & (order_data["status"].isin([1,2]))]
        temp = pick_data[["order_accept_date", "total_amount", "store_name"]].copy()
        temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
        temp.set_index("order_accept_date", inplace=True)
        i += 1
        ax = fig.add_subplot(1, len(val["new"]), i)
        ax.plot(temp.resample("D").sum())
        ax.set_title(trg)

select2 = SelectMultiple(options=store_list)
select2.observe(graph_by_multi, names="value")
display(select2) 

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

### ノック２３：スライドバーを用いてオーダー件数を調べてみよう

In [9]:
from ipywidgets import IntSlider

def store_lower(val):
    clear_output()
    display(slider)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() < val["new"]])

slider = IntSlider(value=1100, min=1000, max=2000, step=100, description="件数:",)
slider.observe(store_lower, names="value")
display(slider)

IntSlider(value=1100, description='件数:', max=2000, min=1000, step=100)

In [10]:
def store_upper(val):
    clear_output()
    display(slider2)
    temp = order_data.groupby("store_name")
    print(temp.size()[temp.size() > val["new"]])

slider2 = IntSlider(value=1600, min=1000, max=2000, step=100, description="件数:",)
slider2.observe(store_upper, names="value")
display(slider2)

IntSlider(value=1600, description='件数:', max=2000, min=1000, step=100)

### ノック２４：トグルボタンで地域データを抽出しよう

In [11]:
from ipywidgets import ToggleButtons

area_list = m_area["wide_area"].unique()

def order_by_area(val):
    clear_output()
    display(toggle)
    pick_data = order_data.loc[(order_data["wide_area"]==val["new"]) & (order_data["status"].isin([1,2]))]
    display(pick_data.head())

toggle = ToggleButtons(options=area_list)
toggle.observe(order_by_area, names="value")
display(toggle)

ToggleButtons(options=('東京', '神奈川', '千葉', '埼玉', '北関東'), value='東京')

In [12]:
def graph_by_area(val):
    clear_output()
    display(toggle2)
    pick_data = order_data.loc[(order_data["wide_area"]==val["new"]) & (order_data["status"].isin([1,2]))]
    temp = pick_data[["order_accept_data", "total_amount"]].copy()
    temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
    temp.set_index("order_accept_date", inplace=True)
    temp.resample("D").sum().plot()

toggle2 = ToggleButtons(options=area_list)
toggle2.observe(graph_by_area, names="value")
display(toggle2)

ToggleButtons(options=('東京', '神奈川', '千葉', '埼玉', '北関東'), value='東京')

### ノック２５：日付を指定してデータを抽出してみよう

In [13]:
from ipywidgets import DatePicker
import datetime

order_data.loc[:, "order_date"] = pd.to_datetime(order_data["order_accept_date"]).dt.date 

def order_by_date(val):
    clear_output()
    display(date_picker)
    pick_data = order_data.loc[(order_data["order_date"]==val["new"]) & (order_data["status"].isin([1,2]))]
    print(len(pick_data))
    display(pick_data.head())

date_picker = DatePicker(value=datetime.datetime(2020,4,1))
date_picker.observe(order_by_date, names="value")
display(date_picker)


DatePicker(value=datetime.datetime(2020, 4, 1, 0, 0))

In [14]:
min_date = datetime.date(2020, 4, 1)
max_date = datetime.date(2020, 4, 30)

def order_between_date():
    clear_output()
    display(date_picker_min)
    display(date_picker_max)
    pick_data = order_data.loc[(order_data["order_date"]>=min_date) & (order_data["order_date"] <= max_date) & (order_data["status"].isin([1,2]))]
    print(len(pick_data))
    display(pick_data.head())

def set_min_date(val):
    global min_date
    min_date = val["new"]
    order_between_date()

def set_max_date(val):
    global max_date
    max_date = val["new"]
    order_between_date()

date_picker_min = DatePicker(value=min_date)
date_picker_min.observe(set_min_date, names="value")
print("最小日付")
display(date_picker_min)
date_picker_max = DatePicker(value=max_date)
date_picker_max.observe(set_max_date, names="value")
print("最大日付")
display(date_picker_max)


最小日付


DatePicker(value=datetime.date(2020, 4, 1))

最大日付


DatePicker(value=datetime.date(2020, 4, 30))

### ノック２６：ストーリーを考えてデータを構築しよう

In [16]:
import glob
import os
current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, "tbl_order_*.csv")
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
    order_temp = pd.read_csv(file)
    print(f'{file}:{len(order_temp)}')
    order_all = pd.concat([order_all, order_temp], ignore_index=True)

order_all = order_all.loc[order_all["store_id"]!=99]

order_all = pd.merge(order_all, m_store, on="store_id", how="left")
order_all = pd.merge(order_all, m_area, on="area_cd", how="left")

order_all.loc[order_all["takeout_flag"]==0, "takeout_name"] = "デリバリー"
order_all.loc[order_all["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"

order_all.loc[order_all["status"]==0, "status_name"] = "受付"
order_all.loc[order_all["status"]==1, "status_name"] = "お支払済"
order_all.loc[order_all["status"]==2, "status_name"] = "お渡し済"
order_all.loc[order_all["status"]==9, "status_name"] = "キャンセル"

order_all.loc[:, "order_date"] = pd.to_datetime(order_all["order_accept_date"]).dt.date 

order_all.groupby(["store_id", "customer_id"])["total_amount"].describe()


/workspaces/python-lm-system-100/chapter-3/tbl_order_202004.csv:233260
/workspaces/python-lm-system-100/chapter-3/tbl_order_202005.csv:241139
/workspaces/python-lm-system-100/chapter-3/tbl_order_202006.csv:233301


count         mean          std     min      25%  \
store_id customer_id                                                     
1        C00244531     14.0  3319.571429   860.262546  1882.0  2515.75   
         C00493736      7.0  3318.142857   922.546915  2471.0  2719.00   
         C01249550      8.0  2664.375000   848.339881  1857.0  2029.75   
         C02241044      6.0  3653.833333  1184.407770  2252.0  2571.00   
         C02859946      9.0  3216.555556   858.216044  1857.0  2647.00   
...                     ...          ...          ...     ...      ...   
999      C98965135     12.0  2867.750000   718.500854  1900.0  2305.50   
         C99065779     19.0  3090.631579  1045.455521   939.0  2310.00   
         C99196263     16.0  3079.250000   964.855119  1900.0  2435.25   
         C99428523     12.0  3350.666667  1373.157432   939.0  2381.25   
         C99813215      9.0  3335.222222  1210.832439   939.0  2732.00   

                         50%      75%     max  
store_id customer_id                           
1        C00244531    3679.5  3901.00  4462.0  
         C00493736    2987.0  3615.50  5100.0  
         C01249550    2379.5  3132.75  3931.0  
         C02241044    3979.5  4683.75  4692.0  
         C02859946    3586.0  3900.00  3931.0  
...                      ...      ...     ...  
999      C98965135    2750.0  3590.75  3901.0  
         C99065779    3050.0  3723.50  4744.0  
         C99196263    2770.0  3244.25  5100.0  
         C99428523    3275.0  4632.75  5100.0  
         C99813215    3582.0  4144.00  4659.0  

[41415 rows x 8 columns]

In [17]:
summary_df = order_all.loc[order_all["status"].isin([1,2])]
store_summary_df = summary_df.groupby(["store_id"])["total_amount"].sum()
store_summary_df = pd.merge(store_summary_df, m_store, on="store_id", how="left")
print("売上上位")
display(store_summary_df.sort_values("total_amount", ascending=False).head(10))
print("売上下位")
display(store_summary_df.sort_values("total_amount", ascending=True).head(10))


売上上位


,store_id,total_amount,store_name,area_cd
47,48,11992501,上野店,TK
189,191,11821224,伊勢崎店,GU
121,123,11320012,栄店,KN
67,68,11272728,八王子店,TK
62,63,11201742,西多摩店,TK
125,127,11179377,戸塚店,KN
98,100,11128515,中原店,KN
43,44,11079128,世田谷店,TK
171,173,11001277,戸田店,SA
78,79,10989669,三鷹店,TK


売上下位


,store_id,total_amount,store_name,area_cd
99,101,5350100,宮前店,KN
75,76,5701753,本郷店,TK
2,3,5712210,足立店,TK
150,152,5723229,四街道店,CH
11,12,5808496,西葛西店,TK
194,196,5901819,前橋店,GU
184,186,6087730,足利店,TO
160,162,6158812,越谷店,SA
69,70,6242227,東久留米店,TK
144,146,6307301,成田店,CH


In [18]:
cancel_df = pd.DataFrame()
cancel_cnt = order_all.loc[order_all["status"]==9].groupby(["store_id"])["store_id"].count()
order_cnt = order_all.loc[order_all["status"].isin([1,2,9])].groupby(["store_id"])["store_id"].count()
cancel_rate = (cancel_cnt / order_cnt) * 100
cancel_df["cancel_rate"] = cancel_rate
cancel_df = pd.merge(cancel_df, m_store, on="store_id", how="left")
print("キャンセル率が低い")
display(cancel_df.sort_values("cancel_rate", ascending=True).head(10))
print("キャンセル率が高い")
display(cancel_df.sort_values("cancel_rate", ascending=False).head(10))


キャンセル率が低い


,store_id,cancel_rate,store_name,area_cd
120,122,16.464415,港北店,KN
129,131,16.641406,我孫子店,CH
118,120,16.702298,金沢店,KN
151,153,17.036115,上尾店,SA
19,20,17.127722,王子店,TK
40,41,17.206367,荻窪店,TK
68,69,17.295981,羽村店,TK
121,123,17.314335,栄店,KN
33,34,17.321016,恵比寿店,TK
180,182,17.322176,那珂店,IB


キャンセル率が高い


,store_id,cancel_rate,store_name,area_cd
7,8,20.305677,板橋店,TK
137,139,20.148791,白井店,CH
130,132,19.948335,市川店,CH
155,157,19.929245,川越店,SA
95,96,19.657880,川崎店,KN
65,66,19.627809,石神井店,TK
75,76,19.534050,本郷店,TK
9,10,19.480519,稲城店,TK
18,19,19.470046,赤羽店,TK
160,162,19.469357,越谷店,SA


### ノック２７：キャンセルの理由を分析してみよう

### ノック２８：仮説を検証してみよう

### ノック２９：ストーリーをもとにパーツやデータを組み合わせてダッシュボードを作ろう

### ノック３０：ダッシュボードを改善しよう